# Volume estimation test

In [3]:
import warnings
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*use @default decorator instead.*')
from mdtraj.geometry.sasa import _ATOMIC_RADII
_ATOMIC_RADII['Cl'] = 0.16

In [4]:
traj = md.load('10v_restarted_trajectory.dcd',top="10v_final.pdb")

/Users/rossg/miniconda2/lib/python2.7/site-packages/mdtraj/formats/pdb/pdbfile.py:194: RuntimeWarning: divide by zero encountered in true_divide
  particle_density = traj.top.n_atoms / traj.unitcell_volumes[0]
/Users/rossg/miniconda2/lib/python2.7/site-packages/mdtraj/formats/pdb/pdbfile.py:196: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn('Unlikely unit cell vectors detected in PDB file likely '


In [5]:
def init_grid(xyz,spacing,padding) :
    """
    Initialize a grid based on a list of x,y,z coordinates. Written by Sam Genheden.

    Parameters
    ----------
    xyz  : Numpy array 
      Cartesian coordinates that should be covered by the grid
    spacing : float 
      the grid spacing
    padding : float 
      the space to add to minimum extent of the coordinates

    Returns
    -------
    Numpy array
      the grid 
    list of Numpy arrays
      the edges of the grid
    """
  
    origin  = np.floor(xyz.min(axis=0))-padding
    tr      = np.ceil(xyz.max(axis=0))+padding
    length  = tr-origin
    shape  =  np.array([int(l/spacing + 0.5) + 1 for l in length],dtype=int)
    grid    = np.zeros(shape)
    edges  = [np.linspace(origin[i],tr[i],shape[i]) for i in range(3)]
    return grid,edges

def _voxel(coord,edges) :
    """
    Wrapper for the numpy digitize function to return the grid coordinates. Written by Sam Genheden.
    """
    return np.array([np.digitize(coord,edges[i])[i] for i in range(3)],dtype=int)

def _fill_sphere(coord,grid,edges,spacing,radius) :
    """
    Fill a grid using spherical smoothing

    Parameters
    ----------
    coord : Numpy array
      the Cartesian coordinates to put on the grid
    grid  : Numpy array
      the 3D grid. Will be modified
    edges : list of Numpy array
      the edges of the grid
    spacing : float 
      the grid spacing
    radius  : float 
      the radius of the smoothing
    """
    # Maximum coordinate
    maxxyz = np.minimum(coord + radius,np.array([edges[0][-1],edges[1][-1],edges[2][-1]]))
    i = 1
    # Iterate over the sphere
    rad2 = radius**2
    x = max(coord[0] - radius,edges[0][0])
    while x <= maxxyz[0] :
        y = max(coord[1] - radius,edges[1][0])
        while y <= maxxyz[1] :
            z = max(coord[2] - radius,edges[2][0])
            while z <= maxxyz[2] :
            # Check if we are on the sphere
                r2 = (x-coord[0])**2+(y-coord[1])**2+(z-coord[2])**2
                if r2 <= rad2 :
                # Increase grid with one
                    v = _voxel(np.array([x,y,z]),edges)
                    grid[v[0],v[1],v[2]] = grid[v[0],v[1],v[2]] + 1
                z = z + spacing
            y = y + spacing
        x = x + spacing
        #print x
        
    
def calc_volume(xyz,extent=1.0,spacing=0.5) :
    
    print("initializing grid")
    grid,edges = init_grid(xyz,spacing,0)
    
    print("Filling sphere")
    for coord in xyz :
        _fill_sphere(coord,grid,edges,spacing,extent)
    print("Calculating volume")    
    volume = np.sum(grid >= 1)*(spacing**3)
    
    return edges, grid, volume

In [174]:
edges, grid, volume = calc_volume(traj.xyz[0],extent=0.5,spacing=0.5)
print volume

initializing grid
Filling sphere
Calculating volume
129.125


In [122]:
counts = np.zeros(len(edges[0])*len(edges[1])*len(edges[2]))
print len(edges[0])*len(edges[1])*len(edges[2])

2805


In [166]:
xyz = traj.xyz[0]

grid,edges = init_grid(xyz,spacing=0.2,padding=0)
            
counts = np.zeros(len(edges[0])*len(edges[1])*len(edges[2]))
xyz = traj.xyz[0]
c = 1
for i in range(len(edges[0])-1):
    for j in range(len(edges[1])-1):
        for k in range(len(edges[2])-1):
            minarr = np.array((edges[0][i],edges[1][j],edges[2][k]))
            maxarr = np.array((edges[0][i+1],edges[1][j+1],edges[2][k+1]))
            counts[c] = np.sum(np.prod(xyz < maxarr,axis=1)*np.prod(xyz > minarr,axis=1))
            c += 1
if counts.sum() != traj.n_atoms: print("Some atoms not accounted for")
vol = (counts >=1).sum()*(spacing**3)  

print vol

458.0


In [167]:
counts.sum()

5300.0

In [168]:
np.sum(np.prod(xyz < maxarr,axis=1))

5300

In [171]:
np.sum(np.prod(xyz > minarr,axis=1))

0

In [172]:
np.sum(np.prod(xyz < maxarr,axis=1)*np.prod(xyz > minarr,axis=1))

0

## Monte carlo estimate of volume:

In [176]:
np.random.uniform(low=0.0, high=1.0, size=1)

array([ 0.44365883,  0.93034467])

In [22]:
def MonteCarloVolume(xyz,N,cutoff):
    high =xyz.max(axis=0)
    low = xyz.min(axis=0)

    xsamps = np.random.uniform(low=low[0],high=high[0], size=N)
    ysamps = np.random.uniform(low=low[1],high=high[1], size=N)
    zsamps = np.random.uniform(low=low[2],high=high[2], size=N)
    
    samps = np.ones((N,3))
    samps[:,0] = xsamps
    samps[:,1] = ysamps
    samps[:,2] = zsamps

    dists = np.zeros(N)
    for i in range(N):
        dists[i] = ((samps[i] - xyz)**2).min()
    #print dists
    return np.sum(dists < cutoff**(0.5))/N

In [26]:
MonteCarloVolume(traj.xyz[0],1000000,0.001)

1

In [13]:
xyz = traj[0].xyz

In [14]:
xyz.min(axis=0)

array([[ 2.8917377 ,  4.20494604,  2.93749046],
       [ 3.04198384,  4.04332161,  2.92347884],
       [ 2.9590981 ,  4.09657288,  3.12257862],
       ..., 
       [ 1.91835046,  1.0110172 ,  2.74777079],
       [ 2.10053682,  1.02576149,  2.62501955],
       [ 2.12009239,  0.91884911,  2.62675333]], dtype=float32)